In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score

In [2]:
def transfer_to(inputs, labels, device):
    return inputs.to(device), labels.to(device)

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/Homeworks FDS/Progetto
%pwd

/content/drive/.shortcut-targets-by-id/1Vzw8m8Ha_VscaXjjIlDjZrQtp1m45M3N/Homeworks FDS/Progetto


'/content/drive/.shortcut-targets-by-id/1Vzw8m8Ha_VscaXjjIlDjZrQtp1m45M3N/Homeworks FDS/Progetto'

# Dataset Initialization

In [5]:
class MyDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.file_paths = []
        self.labels = []

        for label_dir in ['artificial', 'natural']:
            label = 0 if label_dir == 'artificial' else 1
            folder = os.path.join(root_dir, label_dir)
            for filename in os.listdir(folder):
                if filename.endswith(".png") or filename.endswith(".jpg"):
                    self.file_paths.append(os.path.join(folder, filename))
                    self.labels.append(label)

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        label = self.labels[idx]

        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        return image, label

In [6]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

dataset = MyDataset(root_dir='./6k_samples_dataset/dataset', transform=transform)

train_size = int(0.7*len(dataset))
val_size = int(0.2*len(dataset))
test_size = len(dataset)-train_size-val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Model Definition and Initialization

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.block1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )

        self.block3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )

        self.dropout = nn.Dropout(0.5)

        self.fc1 = nn.Linear(128 * 28 * 28, 256)
        self.fc2 = nn.Linear(256, 1)

    def forward(self, x):
        x = self.block1(x)

        x = self.block2(x)

        x = self.block3(x)

        x = x.view(x.size(0), -1)

        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CNN()
model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Train Function

In [9]:
def train_model(model, train_loader, val_loader, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:

            inputs, labels = transfer_to(inputs, labels, device)

            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs.view(-1), labels.float())
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            probabilities = torch.sigmoid(outputs)
            predicted = (probabilities > 0.5).long()
            total += labels.size(0)
            correct += (predicted.view(-1) == labels).sum().item()

        train_accuracy = 100 * correct / total

        val_loss, val_accuracy = evaluate_model(model, val_loader)

        print(f"Epoch {epoch + 1}/{num_epochs}")
        print(f"Train Loss: {train_loss / len(train_loader):.4f}, "
              f"Train Accuracy: {train_accuracy:.2f}%")
        print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%\n")

Evaluate function

In [10]:
def evaluate_model(model, data_loader):
    model.eval()
    loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = transfer_to(inputs, labels, device)

            outputs = model(inputs)

            loss += criterion(outputs.view(-1), labels.float()).item()

            probabilities = torch.sigmoid(outputs)
            predicted = (probabilities > 0.5).long()
            total += labels.size(0)
            correct += (predicted.view(-1) == labels).sum().item()

    accuracy = 100 * correct / total
    return loss / len(data_loader), accuracy

Test function

In [11]:
def test_model(model, test_loader):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = transfer_to(inputs, labels, device)

            outputs = model(inputs)
            test_loss += criterion(outputs.view(-1), labels.float()).item()

            probabilities = torch.sigmoid(outputs)
            predicted = (probabilities > 0.5).long()

            total += labels.size(0)
            correct += (predicted.view(-1) == labels).sum().item()

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.view(-1).cpu().numpy())

    test_loss /= len(test_loader)
    test_accuracy = 100 * correct / total
    precision = precision_score(all_labels, all_predictions)
    recall = recall_score(all_labels, all_predictions)
    f1 = f1_score(all_labels, all_predictions)

    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")

# Model Training and Testing Results

In [12]:
train_model(model, train_loader, val_loader, num_epochs=10)

Epoch 1/10
Train Loss: 2.1663, Train Accuracy: 87.71%
Val Loss: 0.1153, Val Accuracy: 94.92%

Epoch 2/10
Train Loss: 0.1495, Train Accuracy: 94.86%
Val Loss: 0.1628, Val Accuracy: 92.75%

Epoch 3/10
Train Loss: 0.1185, Train Accuracy: 95.98%
Val Loss: 0.3115, Val Accuracy: 87.83%

Epoch 4/10
Train Loss: 0.1222, Train Accuracy: 95.36%
Val Loss: 0.0509, Val Accuracy: 97.67%

Epoch 5/10
Train Loss: 0.0680, Train Accuracy: 97.98%
Val Loss: 0.1648, Val Accuracy: 94.00%

Epoch 6/10
Train Loss: 0.0881, Train Accuracy: 97.33%
Val Loss: 0.1988, Val Accuracy: 94.50%

Epoch 7/10
Train Loss: 0.0500, Train Accuracy: 98.36%
Val Loss: 1.3040, Val Accuracy: 74.42%

Epoch 8/10
Train Loss: 0.0643, Train Accuracy: 98.02%
Val Loss: 0.1242, Val Accuracy: 96.58%

Epoch 9/10
Train Loss: 0.0589, Train Accuracy: 98.07%
Val Loss: 0.0550, Val Accuracy: 97.67%

Epoch 10/10
Train Loss: 0.0541, Train Accuracy: 98.45%
Val Loss: 0.1106, Val Accuracy: 96.42%



In [13]:
test_model(model, test_loader)

Test Loss: 0.0981, Test Accuracy: 95.67%
Precision: 0.9963, Recall: 0.9155, F1-Score: 0.9542
